In [28]:
# parameters and packages
from typing import List, Dict, Tuple

match = 2
mismatch = -1
gap = -2

In [29]:
# helper function: global alignment for similarity score

def lcs_backtrack(v, w, match, mismatch, indel):
    s = []
    for _ in range(len(v) + 1):
        s.append([0] * (len(w) + 1))

    temp = [0]
    for i in range(len(s[0])-1):
        val = temp[i] - indel
        temp.append(val)
    s[0] = temp

    for i in range(1, len(s)):
        val = s[i-1][0] - indel
        s[i][0] = val

    for i in range(1, len(v) + 1):
        for j in range(1, len(w) + 1):
            m = 0
            if v[i-1] == w[j-1]:
                m = match
            else:
                m = -(mismatch)
            s[i][j] = max(s[i-1][j] - indel, s[i][j-1] - indel, s[i-1][j-1] + m)
    return s

# Insert your global_alignment function here, along with any subroutines you need
def similarity(s, t, match_reward=match, mismatch_penalty=mismatch, indel_penalty=gap):
    manhatten = lcs_backtrack(s, t, match_reward, mismatch_penalty, indel_penalty)
    score = manhatten[-1][-1]

    i = len(s)
    j = len(t)
    s1 = ""
    s2 = ""
    while i != 0 and j != 0:
        if manhatten[i][j-1] - indel_penalty == manhatten[i][j]:
            s1 += "-"
            j -= 1
            s2 += t[-1]
            t = t[:-1]
        elif manhatten[i-1][j] - indel_penalty == manhatten[i][j]:
            s1 += s[-1]
            s = s[:-1]
            s2 += "-"
            i -= 1
        else:
            s1 += s[-1]
            s = s[:-1]
            s2 += t[-1]
            t = t[:-1]
            i -= 1
            j -= 1
    return score, s1[::-1], s2[::-1]


In [ ]:
def score_sequence(s, t, lcp):
    score = 0
    # take substrings of s and t to left of lcp, calculate global alignment

    # take substrings of s and t to right of lcp, calculate global alignment

    # return sum of these scores

    return score

def lcp_align(s, t):
    # use lcp array to find longest possible common prefixes and calculate score
    lcps = []
    max_score = float("-inf")
    cur_lcp = None
    for lcp in lcps:
        # find maximum score
        score = score_sequence(s, t, lcp)
        if score > max_score:
            cur_lcp = lcp
    return cur_lcp


def score_transformation(s, t, kmer):
    # use global alignment to score the tandem kmer duplication or deletion


def tandem_transform_lcs(s: str, t: str, k1: int, k2: int, v: int):

    def divide_and_conquer(s, t, k1, k2, v, start, end):

        # find best possible lcp alignement of strings s and t
        common_prefix = lcp_align(s, t)

        # align s and t to common prefix, get the left flanking and right flanking sequence

        # determine search space (all kmers in left or right edge of alignment that can lead to tandem duplication / deletion)
            # # find if there is a tandem duplication and deletion with that respective kmer, if so, it'll be in the search space

        # test all search space values and select the one that leads to highest global alignment score when applied

        # apply transformation

        # similarity score it with ENTIRE s and t string. if pass threshold with global alignemnt, dn this and return s and t transformed

        # if not, apply transformation to s

        # do this recursively with remaining sequence that is before transformation site, if left flanking as well as reamining sequence that is after the transformation site

    
    # you shoudl have a fully transformed tandem s string that when aligned with t, will produce a similarity score above the trheshold v



### TO Delete

In [ ]:
import itertools

def exhaustive_search(kmers, transforms):
    # find all possible combinations of tandem duplications / deletions given set of kmers

    dups = [(kmer[0], kmer[1], "dup") for kmer in kmers]
    dels = [(kmer[0], kmer[1], "del") for kmer in kmers]

    return list(itertools.product(dups+dels, repeat=transforms))

# add string duplication
def duplicate(s, dupl):
    start = dupl[0]
    insert = dupl[1]
    k = len(dupl)

    # insert duplication at next spot that doesn't contain target kmer
    not_inserted = True
    while not_inserted:
        if s[start:start+k] == insert:
            start += k
        else:
            s = s[:start] + insert + s[start:]
            not_inserted = False
    return s

# do tandem string deletion
# def delete(s, delete):


# find tandem transformations
# minimize umber of transforms from s to t

def find_tandem_transformation(s: str, t: str, k1: int, k2: int, v: int):

    # find all possible kmers between lengths k1 and k2 in S
    all_kmers = set()
    for k_len in range(k1,k2+1):
        for i in range(0, len(s)-k_len + 1):
            all_kmers.add((i, s[i:i+k_len]))
    
    # find all possible events
    # assume maximum number of tandem transforms is five, can be adjusted
    t = 5
    for i in range(2, t):
        all_combos = exhaustive_search(all_kmers, i)
        
        # iterate through all possible combinations
        for comb in all_combos:
            transformed_s = s

            # keep track of duplicated transformations
            duplicated = set()

            print(comb)

            # apply transformations to string
            for transforms in comb:
                if transforms[-1] == 'dup':
                    transformed_s = duplicate(transformed_s, transforms)
                    duplicated.add(())
            print(transformed_s)
            
            print()

            break
    # remove any value that has a duplication and deletion in same spot
    # if there's repeated deletions that will destroy teh sequence, bypass this combination too2


find_tandem_transformation("ABC", "ABABAB", 2, 3, 5)


